In [136]:
import pandas as pd
import numpy as np

In [137]:
df = pd.read_csv("sales_data.csv",index_col=0)

In [138]:
df["brand"] = df["brand"].apply(lambda x: x[:-1])

In [139]:
df.head()

,sku,product_line,brand,sales,price
SKU_ID,,,,,
1,Samsung Stereo Headset Wird EO-EG920BWEG,Phones Mobile Accessories,SAMSUNG,1,1359.0
2,HITACHI REF SBS RM-700AGPND4X-(DIA),Refrigerators,HITACHI,1,184408.0
3,SIEMENS DISHWASHER SN26L801IN,Dishwashers,SIEMENS,1,34425.0
4,Super General S/AC 1.5T SGSI185-3BE 3S,Air Conditioners,Super General,1,29750.0
5,Apple 12W USB Power Adapter,Tablets & Detachables,APPLE,1,1700.0


In [140]:
df.info(), df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 1 to 2015
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sku           2015 non-null   object 
 1   product_line  2015 non-null   object 
 2   brand         2015 non-null   object 
 3   sales         2015 non-null   int64  
 4   price         2015 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 94.5+ KB


(None, (2015, 5))

In [141]:
df = df.astype({"sku":"string","product_line":"string","brand":"string"})

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 1 to 2015
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sku           2015 non-null   string 
 1   product_line  2015 non-null   string 
 2   brand         2015 non-null   string 
 3   sales         2015 non-null   int64  
 4   price         2015 non-null   float64
dtypes: float64(1), int64(1), string(3)
memory usage: 94.5 KB


In [143]:
df["product_line"].unique()

<StringArray>
[    'Phones Mobile Accessories',                 'Refrigerators',
                   'Dishwashers',              'Air Conditioners',
         'Tablets & Detachables',               'Gaming Software',
              'Mobile Computing',            'Kitchen Appliances',
                   'MP3 Players',                 'Input Devices',
          'Earphones/Headphones',               'Home Appliances',
              'Washing Machines',               'Water Purifiers',
                    'Microwaves',            'Travel Accessories',
                  'Phones Fixed',       'Smart Phones (OS Based)',
                 'Computer Bags',                  'Shop in Shop',
        'Health & Personal Care',                  'Home Theatre',
                 'Phones Mobile',         'Lifestyle & Wearables',
               'Digital Cameras',                      'Security',
                        'TV LCD',                   'Peripherals',
                 'Storage Media',   'Printers & 

In [144]:
import re

replacement = {
    re.compile(r"&|/"):" ",
    re.compile(r"( )+"):" ",
    re.compile(r"\(|\)"):"",
    re.compile(r"(^ )|( $)"):""
}

df["product_line_clean"] = df["product_line"].str.lower().replace(replacement,regex=True)

In [145]:
df["product_line_clean"].unique()

<StringArray>
[ 'phones mobile accessories',              'refrigerators',
                'dishwashers',           'air conditioners',
        'tablets detachables',            'gaming software',
           'mobile computing',         'kitchen appliances',
                'mp3 players',              'input devices',
       'earphones headphones',            'home appliances',
           'washing machines',            'water purifiers',
                 'microwaves',         'travel accessories',
               'phones fixed',      'smart phones os based',
              'computer bags',               'shop in shop',
       'health personal care',               'home theatre',
              'phones mobile',        'lifestyle wearables',
            'digital cameras',                   'security',
                     'tv lcd',                'peripherals',
              'storage media',  'printers office solutions',
            'gaming hardware',        'imaging accessories',
          

In [272]:
a = df["brand"].unique()
arr = []

for x in a:
    if len(x.strip().split(" ")) > 1:
        arr.append(x)
print(arr)

['Super General', 'Ultimate Ears', 'TRAVEL BLUE', 'Cyber Power', 'ALTEC LANSING', 'EDU SCIENCE', 'BOOK MY SHOW', 'NO SPEC', 'GO PRO', 'FUSE CHICKEN', 'AQUA SURE']


In [147]:
df["brand_lower"] = df["brand"].str.lower()

In [148]:
df.to_parquet("./sales_data.parquet")

In [214]:
def text_to_set(txt,replace=set()):
    if not replace:
        replace = replacement
    for key,val in replace.items():
        txt = re.sub(key,val,txt)
        
    return set(txt.lower().split(" "))

In [172]:
replacement = {
        re.compile(r"&|/|\\"):" ",
        re.compile(r"( )+"):" ",
        re.compile(r"\(|\)|(^ )|( $)"):"",
    }

In [233]:
def exact_match(df,cat,bow):
    ind = df[cat].isin(bow)
    
    if ind.any():
        return df[ind]
    
    return df

In [307]:
txt_set = text_to_set("Samsung Galaxy Mobile Phone",replacement)

temp =exact_match(df,"brand_lower",txt_set)
temp.head()

,sku,product_line,brand,sales,price,product_line_clean,brand_lower
SKU_ID,,,,,,,
1,Samsung Stereo Headset Wird EO-EG920BWEG,Phones Mobile Accessories,SAMSUNG,1,1359.0,phones mobile accessories,samsung
17,Samsung Ref SBS 591L RS554NRUA,Refrigerators,SAMSUNG,1,79815.0,refrigerators,samsung
20,Samsung Wash/Dry 8/6K-WD80J6410,Washing Machines,SAMSUNG,6,54485.0,washing machines,samsung
27,SAMSUNG W/M TL 6KG WA60M4300HD/TL SLV,Washing Machines,SAMSUNG,15,14990.0,washing machines,samsung
28,SAMSUNG REF SBS 604L RS55K5010SL/TL SS,Refrigerators,SAMSUNG,1,72250.0,refrigerators,samsung


In [232]:
def partial_match(df,cat,txt_set):
    
    def match_each(set1,set2):
        for word1 in set1:
            for word2 in set2:
                if word2.startswith(word1):
                    return True
        return False
        
    ind = df[cat].apply(lambda x: match_each(txt_set,text_to_set(x,replacement)))
    if ind.any():
        return df[ind]
    return df

In [308]:
temp =partial_match(temp,"product_line_clean",txt_set)
temp

,sku,product_line,brand,sales,price,product_line_clean,brand_lower
SKU_ID,,,,,,,
1,Samsung Stereo Headset Wird EO-EG920BWEG,Phones Mobile Accessories,SAMSUNG,1,1359.000000,phones mobile accessories,samsung
35,Samsung Galaxy J200G Black,Smart Phones (OS Based),SAMSUNG,15,7297.000000,smart phones os based,samsung
64,Samsung Galaxy J200G Gold,Smart Phones (OS Based),SAMSUNG,30,7335.033333,smart phones os based,samsung
111,Samsung Galaxy S7 Edge Blue,Smart Phones (OS Based),SAMSUNG,1,42900.000000,smart phones os based,samsung
139,SAMSUNG J7 Pro Flip cover Gold,Phones Mobile Accessories,SAMSUNG,1,1104.000000,phones mobile accessories,samsung
...,...,...,...,...,...,...,...
1915,SAMSUNG Dream LED View Cover S8 BLK,Phones Mobile Accessories,SAMSUNG,1,2549.000000,phones mobile accessories,samsung
1918,Samsung S8+ Black,Smart Phones (OS Based),SAMSUNG,9,64900.000000,smart phones os based,samsung
1921,Samsung Galaxy J700F Black,Smart Phones (OS Based),SAMSUNG,4,9949.250000,smart phones os based,samsung


In [309]:
def over_lap(df,cat,txt_set):
    
    return df[df[cat].apply(lambda x: False if len(txt_set.difference(text_to_set(x.lower()))) > 1 else True)]

In [310]:
temp = over_lap(temp,"sku",txt_set)
temp

,sku,product_line,brand,sales,price,product_line_clean,brand_lower
SKU_ID,,,,,,,
